In [3]:
import numpy as np
import plotly.figure_factory as ff

from PageRankSolver.pretty_print import *
import PageRankSolver.fast.solver as fast_solver
import PageRankSolver.fast.tmatrix as fast_tmatrix
import PageRankSolver.slow.solver as solver
import PageRankSolver.slow.tmatrix as tmatrix

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
